In [1]:
import sys
sys.path.append('..')

from models.cnn.trainer import Trainer

c:\Users\iwama\Desktop\Maciej\Eng\meteoapp-data\venv\lib\site-packages\gribapi\__init__.py:23: UserWarning: ecCodes 2.31.0 or higher is recommended. You are running version 2.27.0
  warnings.warn(


In [2]:
trainer = Trainer()

In [3]:
# trainer.load_model('model_state_cnn_fh_1.pt')

In [ ]:
trainer.train(1000)
#done

In [5]:
rmse, y_hat = trainer.evaluate("test")

RMSE for t2m: 1.6919855692010146; MAE for t2m: 1.2696880128825907;
RMSE for sp: 1.3225792603409856; MAE for sp: 0.993548262051931;
RMSE for tcc: 0.2868512455683077; MAE for tcc: 0.19377891355637614;
RMSE for u10: 1.3093819830557705; MAE for u10: 0.9765637953555617;
RMSE for v10: 1.2716058053350747; MAE for v10: 0.9370431075689475;
RMSE for tp: 0.3050391247157081; MAE for tp: 0.08214936769266837;


In [6]:
_ = trainer.evaluate("val")

RMSE for t2m: 1.6752161797182044; MAE for t2m: 1.2708409951410211;
RMSE for sp: 1.4054457217811258; MAE for sp: 1.0278769151959104;
RMSE for tcc: 0.2928279491571064; MAE for tcc: 0.19920485977658642;
RMSE for u10: 1.3292731316291553; MAE for u10: 0.986629246439973;
RMSE for v10: 1.2915481457007367; MAE for v10: 0.9455801061792108;
RMSE for tp: 0.3017690696693449; MAE for tp: 0.08183330512372013;


In [7]:
_ = trainer.evaluate("train")

RMSE for t2m: 1.606069689888612; MAE for t2m: 1.211806523748046;
RMSE for sp: 1.330766810168166; MAE for sp: 0.9875244542227125;
RMSE for tcc: 0.28505570321210866; MAE for tcc: 0.1942108798150933;
RMSE for u10: 1.2453924501900897; MAE for u10: 0.9129533500460814;
RMSE for v10: 1.2201558141859339; MAE for v10: 0.8874124024563651;
RMSE for tp: 0.2662624715507769; MAE for tp: 0.07071644674298815;


In [ ]:
trainer.plot_predictions(data_type='test', pretty=False)

In [9]:
trainer.save_prediction_tensor(y_hat, path='prediction_tensor_cnn_new_real.pt')

In [4]:
rmse_norm, y_hat_norm = trainer.evaluate("test", inverse_norm=False)

RMSE for t2m: 0.20571558239252474; MAE for t2m: 0.154369554154288;
RMSE for sp: 0.0546921972629527; MAE for sp: 0.041084317733544956;
RMSE for tcc: 0.7767842672942566; MAE for tcc: 0.5258929919992062;
RMSE for u10: 0.3934424683654931; MAE for u10: 0.2934368987705869;
RMSE for v10: 0.44426956472907375; MAE for v10: 0.32738303714703604;
RMSE for tp: 1.1005377807336127; MAE for tp: 0.2963824105719532;


In [5]:
trainer.save_prediction_tensor(y_hat_norm, path='prediction_tensor_cnn_new_norm.pt')

In [7]:
import numpy as np
rmse_norm = rmse_norm[0]
np.mean(rmse_norm)

0.49590697679631895